In [7]:
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import pandas as pd
import re
import os
from fake_useragent import UserAgent
import requests

In [94]:
url = 'https://rustih.ru/stihi-russkih-poetov-klassikov/' # website url 

def response(url):
    response = requests.get(url)
    return response

def FindAuthorsHrefs(tree):
    data = []
    
    bsObject = tree.find_all('div', {'class':'taxonomy-description'})
    
    tmp = re.findall("<a(.*?)</a>", str(bsObject[0]), re.DOTALL)
    href = re.findall("href=(.*?) title=", str(tmp), re.DOTALL) #list of hrefs
    
    author = bsObject[0].text.split('\n')
    author = [x for x in author if x != ''] #list of authors
    
    data.append(author)
    data.append(href)
    
    return data

def FixHref(array):
    for i in range(len(array[1])):
        array[1][i] = array[1][i][1:-1]
    return array
    
    
def FindAllPoems(href):
    tree = BeautifulSoup(href.content, 'html.parser')
    return href


def HrefsAllPoems(ListOfHrefs):
    HrefsList = []
    for href in ListOfHrefs:
        tree = BeautifulSoup(response(href).content, 'html.parser')
        for href1 in tree.find_all('span', {'style':'font-size: large;'}):
            #print(href1.a.get('href'))
            HrefsList.append(href1.a.get('href'))
    return HrefsList


def PoemsHrefs(HrefByAuthor):
    AllPoemsHrefs = []
    
    tree = BeautifulSoup(response(HrefByAuthor).content, 'html.parser')
    bsObject = tree.find_all('a', {'class':'title'})
    
    for i in range(len(bsObject)):
        href = re.findall("href=(.*?)>", str(bsObject[i]), re.DOTALL) 
        AllPoemsHrefs.append(str(href)[1:-1])

    return AllPoemsHrefs


def CleanHrefs(hrefs):
    output = []
    for i in hrefs:
        output.append(i[2:-2])
    return output


def PoemToTxt(href, path, flag):  #params : link to a poem, path to download in .txt, download flag
    tree = BeautifulSoup(response(href).content, 'html.parser')
    title = tree.find('h1', {'entry-title'}).text
    text = tree.find('div', {'class': 'entry-content poem-text'}).text
    
    if flag:
        os.chdir(path)
        file = open(title + ".txt", "w+")
        file.write(text)
        file.close()
    return text

**Running**

In [82]:
tree = BeautifulSoup(response(url).content, 'html.parser')
Data = FindAuthorsHrefs(tree) #names and hrefs for them in 2-dim array

In [83]:
Data = FixHref(Data) #preparing hrefs

In [84]:
AllAutorsLinks = HrefsAllPoems(Data[1]) #there we have an array with links to authors

In [85]:
ArrayPoemsHrefs = []

for i in AllAutorsLinks:
    ArrayPoemsHrefs.append(PoemsHrefs(i))

In [86]:
ArrayPoemsHrefs

[['\'"https://rustih.ru/aleksandr-pushkin-19-oktyabrya/"\'',
  '\'"https://rustih.ru/aleksandr-pushkin-19-oktyabrya-1827/"\'',
  '\'"https://rustih.ru/aleksandr-pushkin-27-maya-1819/"\'',
  '\'"https://rustih.ru/aleksandr-pushkin-a-son-amant-egle-sans-resistance/"\'',
  '\'"https://rustih.ru/aleksandr-pushkin-couplets/"\'',
  '\'"https://rustih.ru/aleksandr-pushkin-ex-ungue-leonem/"\'',
  '\'"https://rustih.ru/aleksandr-pushkin-jai-possede-maitresse-honnete/"\'',
  '\'"https://rustih.ru/aleksandr-pushkin-mon-portrait-moj-portret/"\'',
  '\'"https://rustih.ru/aleksandr-pushkin-orlando-furioso/"\'',
  '\'"https://rustih.ru/aleksandr-pushkin-quand-au-front-du-convive/"\'',
  '\'"https://rustih.ru/aleksandr-pushkin-stances/"\'',
  '\'"https://rustih.ru/aleksandr-pushkin-tien-et-mien-dit-lafontaine/"\'',
  '\'"https://rustih.ru/aleksandr-pushkin-a-v-nenastnye-dni/"\'',
  '\'"https://rustih.ru/aleksandr-pushkin-a-shutku-ne-mogu-pridumat-ya-druguyu/"\'',
  '\'"https://rustih.ru/aleksandr-push

In [87]:
ArrayPoemsHrefs = [x for xs in ArrayPoemsHrefs for x in xs] #list of all links to poems (count = 22760)
ArrayPoemsHrefs = CleanHrefs(ArrayPoemsHrefs) #list of all links to poems (count = 22760)

In [88]:
print(len(ArrayPoemsHrefs))

22760


In [118]:
for i in tqdm_notebook(range(0, len(ArrayPoemsHrefs))):
    PoemToTxt(ArrayPoemsHrefs[i], "/Users/kuprone/Desktop/test", True) 

<ipython-input-118-585bc0af6d42>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(21355, len(ArrayPoemsHrefs))):
